# Import Libraries

In [1]:
# Import all the required libraries

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from zipfile import ZipFile 

## Install Lib for analyzing human neurophysiological data

In [2]:
!pip install mne -q

In [3]:
# Make the required imports from MNE
import mne

# Download the dataset

We are using the dataset from the paper :[Reliable and automatic epilepsy classification with affordable, consumer-grade electroencephalography in rural sub-Saharan Africa](https://www.biorxiv.org/content/10.1101/324954v1)<br>
We are using data collected from the Guinea Bissau.

In [4]:
!wget https://zenodo.org/record/1252141/files/EEGs_Guinea-Bissau.zip

--2024-02-27 22:31:06--  https://zenodo.org/record/1252141/files/EEGs_Guinea-Bissau.zip
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1252141/files/EEGs_Guinea-Bissau.zip [following]
--2024-02-27 22:31:07--  https://zenodo.org/records/1252141/files/EEGs_Guinea-Bissau.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 153973086 (147M) [application/octet-stream]
Saving to: 'EEGs_Guinea-Bissau.zip'

EEGs_Guinea-Bissau. 100%[===================>] 146.84M  12.2MB/s    in 14s     

2024-02-27 22:31:21 (10.6 MB/s) - 'EEGs_Guinea-Bissau.zip' saved [153973086/153973086]



# Load data

In [5]:
data = ZipFile('EEGs_Guinea-Bissau.zip')
data.extractall()

In [6]:
metadata=pd.read_csv('https://zenodo.org/record/1252141/files/metadata_guineabissau.csv')
metadata.head()

,subject.id,Group,Eyes.condition,Remarks,recordedPeriod,startTime
0,1,Epilepsy,closed-3min-then-open-2min,by 45s reposition electrodes,301,27/5/2020 14:33
1,2,Control,open-3min-then-closed-2min,NaN,309,26/5/2020 22:44
2,3,Epilepsy,closed-3min-then-open-2min,NaN,309,27/5/2020 14:26
3,4,Epilepsy,closed-3min-then-open-2min,"Green lights not shown, but good EEG traces",299,27/5/2020 15:23
4,5,Control,closed-3min-then-open-2min,NaN,302,23/5/2020 19:09


In [7]:
print("Total number of signal files :", len(metadata))

Total number of signal files : 97


# Preprocessing the data

## Split the dataset into Control and Epilepsy

In [8]:
# Seprate by subject type
Epi_df =metadata['subject.id'][metadata['Group']=='Epilepsy']
Cont_df =metadata['subject.id'][metadata['Group']=='Control']

In [9]:
# Get the csv files

Epilepsy = []
Control = []

for i in  Epi_df:
    Epilepsy.append(pd.read_csv(f'EEGs_Guinea-Bissau/signal-{i}.csv.gz', compression='gzip'))
for i in  Cont_df:
    Control.append(pd.read_csv(f'EEGs_Guinea-Bissau/signal-{i}.csv.gz', compression='gzip'))

In [10]:
pd.set_option('display.max_columns', None)

In [11]:
Epilepsy[0].head(10)

,Unnamed: 0,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,COUNTER,INTERPOLATED,GYROX,GYROY,RAW_CQ,CQ_CMS,CQ_F7,CQ_T7,CQ_O2,CQ_FC6,CQ_AF4,CQ_F3,CQ_P7,CQ_P8,CQ_F4,CQ_AF3,CQ_FC5,CQ_O1,CQ_T8,CQ_F8,CQ_DRL
0,1,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,4297.948718,4186.153846,4132.820513,4106.666667,4076.410256,28,0,1555,1751,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
1,2,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,4295.897436,4187.179487,4126.666667,4105.641026,4065.128205,29,0,1555,1755,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
2,3,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,4301.025641,4188.205128,4123.076923,4103.076923,4063.589744,30,0,1555,1755,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
3,4,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,4298.974359,4184.615385,4127.179487,4095.384615,4071.282051,31,0,1559,1751,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
4,5,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,4295.384615,4182.564103,4128.205128,4090.769231,4064.102564,32,0,1559,1759,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
5,6,4411.282051,3982.051282,4384.102564,3831.282051,4654.358974,3873.333333,4683.076923,3893.333333,4061.025641,4290.769231,4184.615385,4115.384615,4087.692308,4049.743590,33,0,1563,1755,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
6,7,4410.256410,3985.128205,4383.076923,3830.256410,4653.846154,3874.871795,4679.487179,3894.871795,4058.974359,4281.538462,4178.974359,4102.051282,4082.564103,4042.051282,34,0,1559,1755,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
7,8,4403.076923,3976.410256,4377.435897,3822.564103,4654.358974,3869.230769,4680.512821,3893.846154,4049.230769,4278.461538,4169.743590,4105.641026,4081.025641,4044.615385,35,0,1563,1763,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
8,9,4389.743590,3961.538462,4367.179487,3806.666667,4651.794872,3846.666667,4674.871795,3875.384615,4044.615385,4280.512821,4168.205128,4107.692308,4077.435897,4038.974359,36,0,1563,1755,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
9,10,4371.282051,3952.820513,4351.794872,3794.871795,4627.179487,3833.333333,4659.487179,3860.000000,4049.743590,4277.948718,4170.256410,4096.923077,4068.717949,4021.025641,37,0,1563,1755,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


In [12]:
print("Samples in this Epilepsy Signal File: ", len(Epilepsy[0]))

Samples in this Epilepsy Signal File:  38528


In [13]:
print("Samples in this Control Signal File: ", len(Control[0]))

Samples in this Control Signal File:  39552


In [14]:
# Remove SL no. column
Epilepsy=[i.iloc[:,1:15] for i in  Epilepsy]
Control=[i.iloc[:,1:15] for i in  Control]

In [15]:
Epilepsy[0].head()

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8
0,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,4297.948718,4186.153846,4132.820513,4106.666667,4076.410256
1,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,4295.897436,4187.179487,4126.666667,4105.641026,4065.128205
2,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,4301.025641,4188.205128,4123.076923,4103.076923,4063.589744
3,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,4298.974359,4184.615385,4127.179487,4095.384615,4071.282051
4,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,4295.384615,4182.564103,4128.205128,4090.769231,4064.102564


# Load as MNE

In [16]:
def convertDF2MNE(sub):
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128) # Define the info
    info.set_montage('standard_1020') # Define the 10-20 positioning of electrodes
    data=mne.io.RawArray(sub.T, info) # Create Raw. Data passed should be (n_channels,n_times)
    data.set_eeg_reference()
    data.filter(l_freq=0.1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    epochs=epochs.drop_bad()
    
    return epochs

In [17]:
%%capture
# Convert each dataframe to mne object
Epilepsy=[convertDF2MNE(i) for i in  Epilepsy]
Control=[convertDF2MNE(i) for i in  Control]

## Join all Epochs

In [18]:
%%capture
# Concatenate the epochs
Epilepsy_epochs=mne.concatenate_epochs(Epilepsy)
Control_epochs=mne.concatenate_epochs(Control)

## Add labels
0 - Epilepsy
1 - Control

In [19]:
Epilepsy_group=np.concatenate([[i]*len(Epilepsy[i]) for i in range(len(Epilepsy))])#create a list of list where each sub list corresponds to subject_no
Control_group=np.concatenate([[i]*len(Control[i]) for i in range(len(Control))])#create a list of list where each sub list corresponds to subject_no

Epilepsy_label=np.concatenate([[0]*len(Epilepsy[i]) for i in range(len(Epilepsy))])
Control_label=np.concatenate([[1]*len(Control[i]) for i in range(len(Control))])

In [20]:
Epilepsy_group.shape,Control_group.shape,Epilepsy_label.shape,Control_label.shape

((3995,), (3461,), (3995,), (3461,))

In [21]:
# Combine data
data=mne.concatenate_epochs([Epilepsy_epochs,Control_epochs])
group=np.concatenate((Epilepsy_group,Control_group))
label=np.concatenate((Epilepsy_label,Control_label))
print(len(data),len(group),len(label))
     

Not setting metadata
7456 matching events found
No baseline correction applied
7456 7456 7456


In [22]:
type(data[0])

mne.epochs.EpochsArray

# Feature Extraction 
We are going to use power spectral density. It is the distribution of signal power over frequency.

In [23]:
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """
    # specific frequency bands
    
    FREQ_BANDS = {"delta": [0.5, 4.5],
                  "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],
                  "sigma": [11.5, 15.5],
                  "beta": [15.5, 30],
                  "gamma": [30, 45],
                  }

    psds = epochs.compute_psd(method="welch", picks="eeg",fmin=0.5, fmax=45, average='mean')# Compute the PSD using the Welch method
    freqs = psds.freqs
    psds /= np.sum(psds, axis=-1, keepdims=True)
    # Normalize the PSDs
    X = []#For each frequency band, compute the mean PSD in that band
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)# Compute the mean PSD in each frequency band.
        X.append(psds_band)

    return np.concatenate(X, axis=1)#Concatenate the mean PSDs for each band into a single feature vector
 

In [24]:
psds = eeg_power_band(data[0])

Effective window size : 5.000 (s)


In [26]:
%%capture
features=[]
for d in range(len(data)):#get features from each epoch and save in a list
  features.append(eeg_power_band(data[d]))

In [27]:
# We need the list to be a np array
features=np.concatenate(features)
features.shape

(7456, 84)

# Train Model with 5 Fold CV

In [28]:
# do 5 fold cross validation
clf=RandomForestClassifier()
accuracies=cross_val_score(clf, features,label,groups=group,cv=5)
print('Five fold accuracies',accuracies)
print('Average accuracy',np.mean(accuracies))

Five fold accuracies [0.70978552 0.70757881 0.63849765 0.72769953 0.70892019]
Average accuracy 0.6984963399701156
